## Imports and setup

In [ ]:
# Grab data from ACS with cenpy (don't need to run every time!)
# Variables:
#    - total population (B01003_001E)
#    - total white (B02008_001E)
#    - total black (B02009_001E)
#    - total asian (B02011_001E)
#    - total hispanic (B03001_003E)
#    - median household income (B19013_001E)
#    - median house price (B25077_001E)
#    - total tenure (B25032_001E)
#    - total owner occupied (B25032_002E)
#    - total renter occupied (B25032_007E)
#    - median contract rent (B25058_001E)
#    - total in labor force (B23025_002E)
#    - total employed (civilian) (B23025_004E)
#    - total unemployed (civilian) (B23025_005E)

countydata = gpd.GeoDataFrame()

states = pd.read_csv('../utils/states.csv')['State'].values
for state in tqdm(states):
    countydata = countydata.append(acs.from_state(state=state, variables=variables, level='county'))

countydata.to_csv('../data/ACS_countydata.csv')

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import stan_jupyter as stan
from scipy.linalg import norm

from tqdm import tqdm
from cenpy.products import ACS
from spint.gravity import Gravity, Attraction, Production

acs = ACS()
LA_FIPS = '06037'  # LA County FIPS code
variables = ['B01003_001E', 'B02008_001E', 'B02009_001E', 'B02011_001E', 'B03001_003E', \
    'B19013_001E', 'B25077_001E', 'B25032_001E', 'B25032_002E', 'B25032_007E', \
    'B25058_001E', 'B23025_002E', 'B23025_004E', 'B23025_005E']

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

## Load data

In [ ]:
# Flows
inflows_raw = pd.read_csv('../data/LACounty_ACS_2014_2018_All_IN.csv', usecols=['State/County FIPS', 'Total', 'Margin of Error (+/-)']).replace("'", "", regex=True)
outflows_raw = pd.read_csv('../data/LACounty_ACS_2014_2018_All_OUT.csv', usecols=['State/County FIPS', 'Total', 'Margin of Error (+/-)']).replace("'", "", regex=True)
outflows_raw.rename(columns={'State/County FIPS' : 'destFIPS', 'Total' : 'total_out', 'Margin of Error (+/-)' : 'outMOE'}, inplace=True)
inflows_raw.rename(columns={'State/County FIPS' : 'originFIPS', 'Total' : 'total_in', 'Margin of Error (+/-)' : 'inMOE'}, inplace=True)

In [ ]:
# County-level demographic data (and convert covariate columns)
countydata = gpd.read_file('../data/ACS_countydata.csv', GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO").replace('', 0)
for var in variables:
    countydata[var] = countydata[var].astype(float)

In [ ]:
# Ping ACS for data for LA county
la_data = acs.from_county("Los Angeles County, CA", variables=variables, level='county')

In [ ]:
# Convert races, employment to percents
countydata['pctwhite'] = countydata['B02008_001E'] / countydata['B01003_001E']
countydata['pctblack'] = countydata['B02009_001E'] / countydata['B01003_001E']
countydata['pctasian'] = countydata['B02011_001E'] / countydata['B01003_001E']
countydata['pcthispa'] = countydata['B03001_003E'] / countydata['B01003_001E']
countydata['pctemplo'] = countydata['B23025_004E'] / countydata['B23025_002E']
countydata['pctunemp'] = countydata['B23025_005E'] / countydata['B23025_002E']
countydata['pctowner'] = countydata['B25032_002E'] / countydata['B25032_001E']
countydata['pctrentr'] = countydata['B25032_007E'] / countydata['B25032_001E']

la_data['pctwhite'] = la_data['B02008_001E'] / la_data['B01003_001E']
la_data['pctblack'] = la_data['B02009_001E'] / la_data['B01003_001E']
la_data['pctasian'] = la_data['B02011_001E'] / la_data['B01003_001E']
la_data['pcthispa'] = la_data['B03001_003E'] / la_data['B01003_001E']
la_data['pctemplo'] = la_data['B23025_004E'] / la_data['B23025_002E']
la_data['pctunemp'] = la_data['B23025_005E'] / la_data['B23025_002E']
la_data['pctowner'] = la_data['B25032_002E'] / la_data['B25032_001E']
la_data['pctrentr'] = la_data['B25032_007E'] / la_data['B25032_001E']

In [ ]:
# Rename remaining columns and record plaintext variables
countydata.rename(columns={'B25058_001E' : 'medrent', 'B19013_001E' : 'medincome', 'B25077_001E' : 'medhprice'}, inplace=True)
plainvars = ['pctwhite', 'pctblack', 'pctasian', 'pcthispa']
# plainvars = ['pctwhite', 'pctblack', 'pctasian', 'pcthispa', \
#     'medincome', 'medhprice', 'pctowner', 'pctrentr', \
#     'medrent', 'pctemplo', 'pctunemp']
# Problems: medincome, medhprice, medrent: one or more arrays have missing or NaN
#           pctowner, pctrentr, pctemplo, pctunemp: gives NaN SE and tvalues in origin field

## Rearrage data to prep for analysis

In [ ]:
# Merge all the data into one big dataframe to make it simpler
data = pd.merge(pd.merge(countydata, outflows_raw, how='inner', left_on='GEOID', right_on='destFIPS'), inflows_raw, how='inner', left_on='GEOID', right_on='originFIPS').set_crs(epsg=3395)
la_data.rename(columns={'B25058_001E' : 'medrent', 'B19013_001E' : 'medincome', 'B25077_001E' : 'medhprice'}, inplace=True)
la_covars = np.tile(la_data[plainvars].values, (data.shape[0], 1))  # repeat the data so that we account for the single location properly

In [ ]:
# costs are all distances between LA and the out or in destination
coords = np.hstack((data.centroid.x.values.reshape(-1, 1), data.centroid.y.values.reshape(-1, 1)))
la_coords = np.array([la_data.centroid.x[0], la_data.centroid.y[0]])
cost = norm(la_coords - coords, axis=1)

In [ ]:
# Standardize all columns
std_la = (np.exp(la_covars) - np.exp(la_covars).mean()) / np.exp(la_covars).std()
std_data = (np.exp(data[plainvars].values) - np.mean(np.exp(data[plainvars].values))) / np.std(np.exp(data[plainvars].values))

## Calibrate model 

In [ ]:
model = Gravity(flows=data['total_out'].values.reshape(-1, 1), o_vars=std_la, d_vars=std_data, cost=cost, cost_func='exp', constant=False)

In [ ]:
paramnames = [*[x + '_o' for x in plainvars], *[x + '_d' for x in plainvars], 'distance']
pd.DataFrame(data={'paramname' : paramnames, 'paramval' : model.params, 'SE' : model.std_err, 'tvalue' : model.tvalues, 'pvalue' : model.pvalues})  # intercept, origin attrs, dest attrs, distance

In [ ]:
model.SRMSE

## Bayesian code
(just for kicks)

In [ ]:
gravity_code = """
data {
    int<lower=0> n;      // number of flows
    int<lower=1> k_d;    // number of variables at destinations
    int<lower=1> k_o;    // number of variables at origins
    int f[n];            // flows
    matrix[n, k_d] X_d;  // data at destinations
    matrix[n, k_o] X_o;  // data at origins
    vector[n] dist;      // distances for each flow
}

parameters {
    vector[k_o] beta_o;       // origin parameters
    vector[k_d] beta_d;       // destination parameters
    real<upper=0> beta_dist;  // distance decay parameter
    real alpha;               // intercept
}

transformed parameters {
    vector[n] mu = exp(alpha + log(X_o)*beta_o + log(X_d)*beta_d - beta_dist*log(dist));  // mean of flow process
}

model {
    f ~ poisson(mu);

    // priors -- uninformed for everything but beta_dist
    beta_o ~ normal(0, 5);
    beta_d ~ normal(0, 5);
    beta_dist ~ normal(0, 1);
    alpha ~ normal(0, 5);
}

generated quantities {

}
"""

In [ ]:
gravity_data = {
    "n" : data['total_out'].shape[0],
    "f" : data['total_out'].values,
    "k_d" : std_la.shape[1],
    "k_o" : std_data.shape[1],
    "X_d" : std_la,
    "X_o" : std_data,
    "dist" : cost
}

gravity_posterior = stan.build(gravity_code, data=gravity_data)
gravity_fit = gravity_posterior.sample(num_chains=3, num_warmup=9000, num_samples=1000, save_warmup=False)
gravity_results = gravity_fit.to_frame()

## Constrained models

In [ ]:
out_model = Production(flows=data['total_out'].values, origins=data['originFIPS'].values, d_vars=data[variables].values, cost=cost, cost_func='exp')
in_model = Attraction(flows=data['total_in'].values, destinations=data['destFIPS'].values, o_vars=data[variables].values, cost=cost, cost_func='exp')